# 1 列表

## 1.1 将序列分解成单独的变量
问题：现在有一个包含 N 个元素的元组或者是序列，怎样将它里面的值解压后同时赋值给 N 个变量？

解答：可以直接通过解压赋值语句赋值，唯一的前提就是变量的数量必须跟序列元素的数量是一样的，否则会抛异常

In [5]:
data = [ 'ACME', 50, 91.1, (2012, 12, 21) ]
name, shares, price, date = data

name
shares
price
date

(2012, 12, 21)

实际上，这种解压赋值可以用在任何可迭代对象上面，而不仅仅是列表或者元组。
包括字符串，文件对象，迭代器和生成器。

In [6]:
s = 'Hello'
a, b, c, d, e = s
a
b
c
d
e



'o'

如果只想解压一部分，丢掉其他值，可以使用占位符

In [1]:
data = [ 'ACME', 50, 91.1, (2012, 12, 21) ]
_, shares, price, _ = data
shares
price

91.1

## 1.2 解压可迭代对象赋值给多个变量
问题：如果一个可迭代对象的元素个数超过变量个数时，会抛出一个 ValueError 。那么怎样才能从这个可迭代对象中解压出 N 个元素出来？

解答：在变量前加一个"*"，这个变量是列表类型（不管有多少个元素）

In [4]:
import numpy as np
grades = [3, 5, 7, 8, 9]
first, *mid, last = grades
print(np.mean(mid))

6.666666666666667

In [7]:
items = [1, 10, 7, 4, 5, 9]
head, *tail = items
head
tail

[10, 7, 4, 5, 9]

在迭代元素为可变长元组的序列时，星号表达式的作用：


In [5]:
records = [ ('foo', 1, 2),
            ('bar', 'hello'),
            ('foo', 3, 4),
            ]

def do_foo(x, y):
    print('foo', x, y)
def do_bar(s):
    print('bar', s)
for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


如果想解压一些元素后丢弃它们，你不能简单就使用 *，而是"*_"



In [6]:
record = ('ACME', 50, 123.45, (12, 18, 2012))
name, *_, (*_, year) = record
name
year

2012

## 1.3 保留最后 N 个元素
问题：在迭代操作或者其他操作的时候，怎样只保留最后有限几个元素的历史记录？

解答：使用 deque(maxlen=N) 构造函数会新建一个固定大小的队列。当新的元素加入并且这个队列已满的时候，最老的元素会自动被移除掉。

In [9]:
from collections import deque
q = deque(maxlen=3)
q.append(1)
q.append(2)
q.append(3)
print(q)

deque([1, 2, 3], maxlen=3)


In [10]:
q.append(4)
q.append(5)
print(q)


deque([3, 4, 5], maxlen=3)


如果不设置最大队列大小，那么就会得到一个无限大小队列，可以在队列的两端执行添加和弹出元素的操作。

In [11]:
q = deque()
q.append(1)
q.append(2)

In [12]:
q.pop()
q

deque([1])

In [13]:
q.appendleft(5)
q

deque([5, 1])

注意：在队列两端插入或删除元素时间复杂度都是 O(1) ，而在列表的开头插入或删除元素的时间复杂度为 O(N) 。

## 1.4 查找最大或最小的 N 个元素
问题：怎样从一个集合中获得最大或者最小的 N 个元素列表？



* 解答：使用 heapq 模块的两个函数：nlargest() 和 nsmallest()
* 它在底层实现里面，首先会先将集合数据进行堆排序后放入一个列表中，再一个个弹出顶层的元素

In [14]:
import heapq
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
print(heapq.nlargest(3, nums))
print(heapq.nsmallest(3, nums))

[42, 37, 23]
[-4, 1, 2]


两个函数都能接受一个关键字参数，用于更复杂的数据结构中：


In [16]:
portfolio = [ {'name': 'IBM', 'shares': 100, 'price': 91.1},
              {'name': 'AAPL', 'shares': 50, 'price': 543.22},
              {'name': 'FB', 'shares': 200, 'price': 21.09},
              {'name': 'HPQ', 'shares': 35, 'price': 31.75},
              {'name': 'YHOO', 'shares': 45, 'price': 16.35},
              {'name': 'ACME', 'shares': 75, 'price': 115.65} ]
cheap = heapq.nsmallest(3, portfolio, key=lambda s: s['price'])
expensive = heapq.nlargest(3, portfolio, key=lambda s: s['price'])

print(cheap)
print(expensive)

[{'name': 'YHOO', 'shares': 45, 'price': 16.35}, {'name': 'FB', 'shares': 200, 'price': 21.09}, {'name': 'HPQ', 'shares': 35, 'price': 31.75}]
[{'name': 'AAPL', 'shares': 50, 'price': 543.22}, {'name': 'ACME', 'shares': 75, 'price': 115.65}, {'name': 'IBM', 'shares': 100, 'price': 91.1}]


注意：
* 如果你仅仅想查找唯一的最小或最大 (N=1) 的元素的话，那么使用 min() 和max() 函数会更快些
* 如果 N 的大小和集合大小接近的时候，通常先排序这
个集合然后再使用切片操作会更快点 ( sorted(items)[:N] 或者是 sorted(items)[-
N:] )

## 1.5 查找出现次数最多的topK元素
问题：怎样找出一个序列中出现次数最多的几个元素呢？

解决：collections.Counter 类就是专门为这类问题而设计的，它甚至有一个有用的most_common() 方法直接给了你答案

In [10]:
words = [ 'look', 'into', 'my', 'eyes', 'look', 'into', 'my', 'eyes', 'the', 'eyes', 'the', 'eyes', 'the', 'eyes', 'not', 'around', 'the', 'eyes', "don't", 'look', 'around', 'the', 'eyes', 'look', 'into', 'my', 'eyes', "you're", 'under' ]
from collections import Counter
word_counts = Counter(words)
print(word_counts)
# 出现频率最高的 3 个单词
top_three = word_counts.most_common(3)
print(top_three)

Counter({'eyes': 8, 'the': 5, 'look': 4, 'into': 3, 'my': 3, 'around': 2, 'not': 1, "don't": 1, "you're": 1, 'under': 1})
[('eyes', 8), ('the', 5), ('look', 4)]


一个 Counter 对象就是一个字典，可以做并集、差集等操作

In [12]:
morewords = ['why','are','you','not','looking','in','my','eyes']

a = Counter(words)
b = Counter(morewords)

print(a)
print(b)

# Combine counts
c = a + b
print(c)

# Subtract counts
d = a - b
print(d)


Counter({'eyes': 8, 'the': 5, 'look': 4, 'into': 3, 'my': 3, 'around': 2, 'not': 1, "don't": 1, "you're": 1, 'under': 1})
Counter({'why': 1, 'are': 1, 'you': 1, 'not': 1, 'looking': 1, 'in': 1, 'my': 1, 'eyes': 1})
Counter({'eyes': 9, 'the': 5, 'look': 4, 'my': 4, 'into': 3, 'not': 2, 'around': 2, "don't": 1, "you're": 1, 'under': 1, 'why': 1, 'are': 1, 'you': 1, 'looking': 1, 'in': 1})
Counter({'eyes': 7, 'the': 5, 'look': 4, 'into': 3, 'my': 2, 'around': 2, "don't": 1, "you're": 1, 'under': 1})


## 1.6 字典列表排序
问题：有一个字典列表，想根据某个或某几个字典字段来排序这个列表。

解决：使用 sorted 方法，选择指定的 key 进行排序

In [2]:
rows = [ {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
         {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
         {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
         {'fname': 'Big', 'lname': 'Jones', 'uid': 1004} 
         ]

from operator import itemgetter
rows_by_fname = sorted(rows, key=itemgetter('fname'))
rows_by_uid = sorted(rows, key=itemgetter('uid'))
print(rows_by_fname)
print(rows_by_uid)

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]
[{'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}]


itemgetter() 函数支持多个keys，以便二次排序：

In [14]:
rows_by_lfname = sorted(rows, key=itemgetter('lname','fname'))
print(rows_by_lfname)

[{'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}]


也可以使用lambda表达式来替换 itemgetter()：

In [3]:
rows_by_fname = sorted(rows, key=lambda r: r['fname'])
print(rows_by_fname)

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]


注意：这个技术也同样适用于 min() 和 max() 等函数。

## 1.7 排序不支持原生比较的对象
问题：对于自定义类等实例化等对象，如何排序？
解决：（方法一）使用 attrgetter()

In [1]:
class User:
    def __init__(self, user_id):
        self.user_id = user_id
    def __repr__(self):
        return 'User({})'.format(self.user_id)

users = [User(23), User(3), User(99)]
print(users)

from operator import attrgetter
print(sorted(users, key=attrgetter('user_id')))

[User(23), User(3), User(99)]
[User(3), User(23), User(99)]


（方法二）使用 lambda 表达式

In [ ]:
users = [User(23), User(3), User(99)]
print(users)
print(sorted(users, key=lambda u: u.user_id))

注意：attrgetter() 函数通常会运行的快点，并且还能同时允许多个字段进行比较。

## 1.8 通过某个字段将记录分组
问题：有一个字典或者实例的序列，然后你想根据某个特定的字段比如 date 来分组迭代访问
解决：首先需要按照指定的字段 (这里就是 date ) 排序，然后调用 itertools.groupby() 函数分组

In [2]:
rows = [ {'address': '5412 N CLARK', 'date': '07/01/2012'},
         {'address': '5148 N CLARK', 'date': '07/04/2012'},
         {'address': '5800 E 58TH', 'date': '07/02/2012'},
         {'address': '2122 N CLARK', 'date': '07/03/2012'},
         {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'},
         {'address': '1060 W ADDISON', 'date': '07/02/2012'},
         {'address': '4801 N BROADWAY', 'date': '07/01/2012'},
         {'address': '1039 W GRANVILLE', 'date': '07/04/2012'},
]

from operator import itemgetter
from itertools import groupby
# Sort by the desired field first
rows.sort(key=itemgetter('date'))
# Iterate in groups
for date, items in groupby(rows, key=itemgetter('date')):
    print(date)
    for i in items:
        print(' ', i)

07/01/2012
  {'address': '5412 N CLARK', 'date': '07/01/2012'}
  {'address': '4801 N BROADWAY', 'date': '07/01/2012'}
07/02/2012
  {'address': '5800 E 58TH', 'date': '07/02/2012'}
  {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'}
  {'address': '1060 W ADDISON', 'date': '07/02/2012'}
07/03/2012
  {'address': '2122 N CLARK', 'date': '07/03/2012'}
07/04/2012
  {'address': '5148 N CLARK', 'date': '07/04/2012'}
  {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}


注意：在聚合之前必须进行排序，因为 groupby() 仅仅检查连续的元素，如果事先并没有排序完成的话，分组函数将得不到想要的结果

## 1.9 过滤序列元素
问题：有一个数据序列，想利用一些规则从中提取出需要的值或者是缩短序列
（方法一）使用列表推导：

In [5]:
mylist = [1, 4, -5, 10, -7, 2, 3, -1]
print([n for n in mylist if n > 0])
print([n for n in mylist if n < 0])

[1, 4, 10, 2, 3]
[-5, -7, -1]


列表推导等缺点是：如果输入非常大的时候会产生一个非常大的结果集，占用大量内存
（方法二）使用迭代：

In [6]:
for x in mylist:
    if x > 0:
        print(x)

1
4
10
2
3


（方法三）使用自定义filter函数

In [7]:
values = ['1', '2', '-3', '-', '4', 'N/A', '5']
def is_int(val):
    try:
        x = int(val)
        return True
    except ValueError:
        return False
    
ivals = list(filter(is_int, values))
print(ivals)

['1', '2', '-3', '4', '5']


注意：列表推导和生成器表达式通常情况下是过滤数据最简单的方式。
- 它们还能在过滤的时候转换数据
- 也可以用新的值代替，而不是丢弃它们

In [4]:
mylist = [1, 4, -5, 10, -7, 2, 3, -1]
import math
print([math.sqrt(n) for n in mylist if n > 0])
print([n if n > 0 else 0 for n in mylist])


[1.0, 2.0, 3.1622776601683795, 1.4142135623730951, 1.7320508075688772]
[1, 4, 0, 10, 0, 2, 3, 0]


## 1.10 映射名称到序列元素
问题：有一段通过下标访问列表或者元组中元素的代码，但是这样有时候会使得代码难以阅读，应该怎样通过名称来访问元素？
解决：使用 collections.namedtuple() 函数
- 需要传递一个类型名和需要的字段给它，然后它就会返回一个类
- 然后可以初始化这个类，为定义的字段传递值等
- 命名元组的一个主要用途是避免通过下标操作元组，从而提升代码等可读性

In [3]:
from collections import namedtuple
Subscriber = namedtuple('Subscriber', ['addr', 'joined'])
sub = Subscriber('jonesy@example.com', '2012-10-19')
print(sub)
print(sub.addr)
print(sub.joined)

Subscriber(addr='jonesy@example.com', joined='2012-10-19')
jonesy@example.com
2012-10-19


命名元组与字典的区别：
- 字典存储需要更多的内存空间
- 命名元组是不可更改的

In [5]:
Stock = namedtuple('Stock', ['name', 'shares', 'price'])
s = Stock('ACME', 100, 123.45)
print(s)

Stock(name='ACME', shares=100, price=123.45)


AttributeError: can't set attribute

In [ ]:
s.shares = 75
print(s)

使用 _replace() 方法更新

In [7]:
s = s._replace(shares=75)
print(s)

Stock(name='ACME', shares=75, price=123.45)


## 1.11 转换并同时计算数据
问题：需要在数据序列上执行聚集函数 (比如 sum() , min() , max() )，但是首先需要先转换或者过滤数据
解决：使用一个生成器表达式参数

In [4]:
nums = [1, 2, 3, 4, 5] 
s = sum(x * x for x in nums)
print(s)

# 以下方式会创建一个仅仅被使用一次就被丢弃的临时数据结构，不要这样写
s = sum([x * x for x in nums])
print(s)


55


In [2]:
s = ('ACME', 50, 123.45)
print(','.join(str(x) for x in s))

ACME,50,123.45


In [6]:
portfolio = [ {'name':'GOOG', 'shares': 50},
              {'name':'YHOO', 'shares': 75},
              {'name':'AOL', 'shares': 20},
              {'name':'SCOX', 'shares': 65} ]

min_shares = min(s['shares'] for s in portfolio)
print(min_shares)

min_shares = min(portfolio, key=lambda s: s['shares'])
print(min_shares)


20
{'name': 'AOL', 'shares': 20}
